In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
link = 'http://www.fapeg.go.gov.br/categoria/editais/'
parser_general = parser(link)
numpags = parser_general.xpath('//a[@class="page-numbers"]/text()')
numpags = int(numpags[-1])
numpags

titulos_proyectos = []
links_proyectos = []
fechas_proyectos = []
n = 1
while n <= numpags:

    link_pag = link + 'page/' + str(n) + '/'
    

    parser_pag = parser(link_pag)

    title = parser_pag.xpath('//h1[@class="entry-title"]/a/text()')

    link_proy = parser_pag.xpath('//h1[@class="entry-title"]/a/@href')

    date = parser_pag.xpath('//time[@class="entry-date"]/text()')

    titulos_proyectos += title
    links_proyectos += link_proy
    fechas_proyectos += date

    n+=1 

In [5]:
pdfs_proyectos = []


for link in links_proyectos:
    
    
    parser_proyecto = parser(links_proyectos[0])

    pdfs_brutos= parser_proyecto.xpath('//a/@href')

    pdfs_proyecto = ''

    for pdf in pdfs_brutos:
        if pdf.endswith('.pdf'):

            pdfs_proyecto = pdfs_proyecto + pdf + ', '
        
    pdfs_proyectos.append(pdfs_proyecto.strip(', '))


In [8]:
fapeg = pd.DataFrame()

fapeg['Título'] = titulos_proyectos
fapeg['Link'] = links_proyectos
fapeg['Fecha'] = fechas_proyectos
fapeg['Pdfs'] = pdfs_proyectos

fapeg.to_excel('brasil_Fapeg.xlsx')